### installer  un env conda avec python 3.9.24 et langchain installer 

1) Créer l'env conda avec Python 3.9.24 <br>
conda create -n chunking_env python=3.9.24 -y
 <br>
2) Activer l'env <br>
conda activate chunking_env
 <br>
3) Mettre à jour pip <br>
python -m pip install --upgrade pip
 <br>
4) Installer langchain et splitters (et ipykernel pour Jupyter) <br>
pip install langchain langchain-text-splitters ipykernel
 <br>
5) Enregistrer un kernel Jupyter pour `chunking.ipynb` <br>
python -m ipykernel install --user --name chunking_env --display-name "chunking (py3.9.24)"
 <br>
6) Vérifier l'installation <br>
python -c "import langchain; print('langchain', getattr(langchain,'__version__', 'version unknown'))" <br>

In [31]:
import os
import re

# ==========================================
# 🎓 COURS PYTHON : LES 7 STRATÉGIES DE CHUNKING
# Auteur : Crépin & Gemini
# Approche : "From Scratch" (Sans librairies externes)
# ==========================================

def lire_fichier(chemin):
    """Lit un fichier et retourne son contenu."""
    try:
        with open(chemin, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return ""


In [32]:

def afficher_chunks(titre_strategie, chunks):
    """Fonction utilitaire pour visualiser joliment les résultats."""
    print(f"\n{'='*60}")
    print(f"🛠️  STRATÉGIE : {titre_strategie}")
    print(f"{'='*60}")
    print(f"📊 Nombre de morceaux générés : {len(chunks)}")
    print("-" * 60)
    
    for i, chunk in enumerate(chunks):
        # Nettoyage pour l'affichage (preview sur une ligne)
        preview = chunk.replace('\n', ' ').strip()[:80] + "..." 
        print(f"🔹 [Chunk {i+1}] (Taille: {len(chunk)} chars)")
        print(f"   Contenu : \"{preview}\"")
    print("-" * 60)


Stratégie 1 : Le Découpage Fixe (taille fixe)
L’idée : Couper une baguette exactement tous les 10 centimètres.

L’explication : On prend le texte et on le coupe méchamment tous les 200 caractères. C’est comme si tu donnais une baguette à quelqu’un et qu’il la coupait au couteau sans regarder. Avantage : C’est très rapide et simple. Inconvénient : On peut couper un mot en deux (par exemple « informa-tion »). C’est moche et difficile à lire pour une intelligence artificielle.

In [33]:

# ==========================================
# 1️⃣ FIXED-SIZE CHUNKING (Découpage Fixe)
# ==========================================
def chunking_fixe(texte, taille_chunk=200):
    words = texte.split() # On découpe par mots pour éviter de couper un mot en deux
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        if current_length + len(word) + 1 > taille_chunk:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word)
        else:
            current_chunk.append(word)
            current_length += len(word) + 1
            
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks


Stratégie 2 : Le Chevauchement (Chevauchement)
L’idée : Faire des photocopies avec une marge de sécurité.

L’explication : Au lieu de couper net, on fait des morceaux qui se chevauchent.
Imaginez une copie double. La page 1 finit à 50 % de la feuille, et la page 2 recommence 20 % avant la fin de la page 1. Comme ça, si une phrase importante est à la coupure, elle est présente sur les deux morceaux. On est sûr de ne rien perdre.

In [34]:


# ==========================================
# 2️⃣ OVERLAPPING CHUNKING (Avec Chevauchement)
# ==========================================
def chunking_overlap(texte, taille_chunk=200, overlap=50):
    chunks = []
    step = taille_chunk - overlap
    if step <= 0: step = 1 # Sécurité
    
    for i in range(0, len(texte), step):
        chunks.append(texte[i:i + taille_chunk])
        if i + taille_chunk >= len(texte):
            break
    return chunks


Stratégie 3 : Le Découpage Sémantique (sémantique)
L’idée : Couper quand ça a du sens, c’est-à-dire à la fin des phrases.

L’explication : Ici, on ne regarde pas la taille, on regarde la ponctuation. On attend de voir un point « . », un « ! » ou un « ? » pour couper. C’est comme si tu découpais un texte article par article. Chaque morceau est une phrase complète. C’est beaucoup mieux pour comprendre l’histoire, mais les morceaux peuvent être très inégaux (une phrase de 3 mots, une autre de 50 mots).

In [35]:

# ==========================================
# 3️⃣ SEMANTIC CHUNKING (Sémantique / Phrases)
# ==========================================
def chunking_semantique(texte):
    """Découpe en respectant les phrases (finissant par . ! ?)"""
    # Regex simple pour séparer les phrases
    phrases = re.split(r'(?<=[.!?])\s+', texte)
    return [p for p in phrases if p.strip()]


Stratégie 4 : Le Découpage Hiérarchique (Hierarchique)
L’idée : Respecter les titres du livre.

L’explication : Tu regardes la structure du texte (comme dans un livre avec des Titres 1, Titres 2...). Dès que tu vois un titre (« Chapitre 1 »), tu coupes juste avant. Tu crées ainsi des paquets : « Tout ce qui appartient au Chapitre 1 », « Tout ce qui appartient au Chapitre 2 ». C’est hyper pratique pour s’y retrouver.

In [36]:

# ==========================================
# 4️⃣ HIERARCHICAL CHUNKING (Hiérarchique)
# ==========================================
def chunking_hierarchique(markdown_text):
    """Sépare d'abord par titres (H1, H2), puis chunk le contenu."""
    chunks = []
    # On split sur les titres Markdown (# Titre)
    sections = re.split(r'(^#+ .*$)', markdown_text, flags=re.MULTILINE)
    
    titre_courant = "Introduction"
    buffer_texte = ""

    for item in sections:
        if item.strip().startswith('#'):
            # C'est un nouveau titre, on sauvegarde la section précédente
            if buffer_texte.strip():
                chunks.append(f"SECTION: {titre_courant}\nCONTENU: {buffer_texte.strip()}")
            titre_courant = item.strip()
            buffer_texte = ""
        else:
            buffer_texte += item
            
    # Ajouter la dernière section
    if buffer_texte.strip():
        chunks.append(f"SECTION: {titre_courant}\nCONTENU: {buffer_texte.strip()}")
    
    return chunks


Stratégie 5 : Le Découpage Récursif (Récursif)
L’idée : Le couteau intelligent qui essaye d’être poli avant d’être brutal.

L’explication : C’est la méthode la plus intelligente de la partie manuelle. Le robot se dit :

« Est-ce que je peux couper entre les paragraphes (saut de ligne) ? »
Si c’est trop grand, « Est-ce que je peux couper entre les phrases ? »
Si c’est trop grand, « Est-ce que je peux couper entre les mots ? » Il essaye de garder le texte le plus lisible possible avant de couper n’importe où.

In [37]:

# ==========================================
# 5️⃣ RECURSIVE CHUNKING (Récursif)
# ==========================================
def chunking_recursif(texte, taille_max=300):
    """Essaie de couper par \n\n, puis \n, puis espace."""
    if len(texte) <= taille_max:
        return [texte]
        
    separateurs = ["\n\n", "\n", ". ", " "]
    
    for sep in separateurs:
        if sep in texte:
            morceaux = texte.split(sep)
            chunks = []
            chunk_temp = ""
            
            for m in morceaux:
                if len(chunk_temp) + len(m) < taille_max:
                    chunk_temp += m + sep
                else:
                    if chunk_temp: chunks.append(chunk_temp)
                    chunk_temp = m + sep
            if chunk_temp: chunks.append(chunk_temp)
            
            # Vérification : si on a réussi à réduire la taille, on retourne
            # Sinon, on passe au séparateur suivant (plus fin)
            return chunks
            
    return [texte] # Fallback si impossible de couper


Stratégie 6 : Le Découpage Adaptatif (Adaptive)
L’idée : Changer d’outil selon ce qu’on coupe.

L’explication : C’est comme un chef cuisinier qui regarde son ingrédient.

« Tiens, c’est du code HTML compliqué. » -> Il prend un petit couteau précis (découpage fixe petit).
« Tiens, c’est un long texte. » -> Il utilise le couteau récursif (vu juste avant).
Le code analyse le texte et décide tout seul de la meilleure méthode.

In [38]:

# ==========================================
# 6️⃣ ADAPTIVE CHUNKING (Adaptatif)
# ==========================================
def chunking_adaptatif(texte):
    """Choisit la méthode selon le contenu."""
    # Si ça ressemble à du HTML/Code
    if "<html>" in texte or "def " in texte:
        return ["MODE CODE/HTML ACTIVÉ"] + chunking_fixe(texte, 100)
    
    # Si c'est un texte court
    if len(texte) < 500:
        return chunking_semantique(texte)
    
    # Sinon, méthode robuste récursive
    return chunking_recursif(texte, 300)


Stratégie 7 : Le Découpage Structurel (HTML structural)
L’idée : Extraire la viande du sandwich HTML.

L’explication : Une page web est pleine de codes bizarres (, ). Cette méthode ignore les codes et garde juste le texte contenu dedans. C’est comme peler les épluchures d’une pomme pour ne manger que le fruit.<div><h1>

In [39]:

# ==========================================
# 7️⃣ STRUCTURAL CHUNKING (Structurel HTML)
# ==========================================
def chunking_structurel(html_text):
    """Extrait le contenu basé sur les balises HTML (h1, p, div)"""
    # Regex simple pour capturer le contenu entre balises
    # Note: En prod, utiliser BeautifulSoup est mieux, mais ici on fait "from scratch"
    pattern = r"<(h[1-6]|p|div)>(.*?)</\1>"
    matches = re.findall(pattern, html_text, re.DOTALL)
    
    chunks = []
    for tag, content in matches:
        clean_content = re.sub(r'<.*?>', '', content).strip() # Enlève les balises imbriquées
        if clean_content:
            chunks.append(f"TYPE: {tag.upper()} | CONTENT: {clean_content}")
            
    return chunks


# PARTIE 2 : LA MÉTHODE PROFESSIONNELLE (LangChain)
Ici, on arrête de bricoler. On utilise des outils industriels, testés et optimisés par des experts.

In [40]:

# ==========================================
# 🚀 EXÉCUTION
# ==========================================

print("📂 Chargement des données...")
md_content = lire_fichier("data/histoire_ia.md")
html_content = lire_fichier("data/page_web.html")



# Tests des 7 méthodes
afficher_chunks("1. FIXED-SIZE", chunking_fixe(md_content, 200))
afficher_chunks("2. OVERLAPPING", chunking_overlap(md_content, 200, 50))
afficher_chunks("3. SEMANTIC (Phrases)", chunking_semantique(md_content))
afficher_chunks("4. HIERARCHICAL (Markdown)", chunking_hierarchique(md_content))
afficher_chunks("5. RECURSIVE", chunking_recursif(md_content, 300))

print("\n--- TEST ADAPTATIF ---")
afficher_chunks("6. ADAPTIVE (Sur texte MD)", chunking_adaptatif(md_content))
afficher_chunks("6. ADAPTIVE (Sur Code HTML)", chunking_adaptatif(html_content))

afficher_chunks("7. STRUCTURAL (Extraction HTML)", chunking_structurel(html_content))



📂 Chargement des données...

🛠️  STRATÉGIE : 1. FIXED-SIZE
📊 Nombre de morceaux générés : 8
------------------------------------------------------------
🔹 [Chunk 1] (Taille: 198 chars)
   Contenu : "L'Histoire et l'Évolution de l'Intelligence Artificielle 1. Les Fondations (1950..."
🔹 [Chunk 2] (Taille: 196 chars)
   Contenu : "and Intelligence". Il y pose la question fondamentale : "Les machines peuvent-el..."
🔹 [Chunk 3] (Taille: 191 chars)
   Contenu : "battre un champion d'échecs en quelques années. C'était l'époque de l'IA symboli..."
🔹 [Chunk 4] (Taille: 199 chars)
   Contenu : "financements se sont taris. Les promesses n'ont pas été tenues. C'est ce qu'on a..."
🔹 [Chunk 5] (Taille: 200 chars)
   Contenu : "et Big Data (1990-2010) Avec l'arrivée d'internet, la quantité de données a expl..."
🔹 [Chunk 6] (Taille: 199 chars)
   Contenu : "Learning (Réseaux de neurones profonds) grâce aux cartes graphiques (GPU). 3. L'..."
🔹 [Chunk 7] (Taille: 199 chars)
   Contenu : "Claude ou Gemin

## 🏗️ C’est quoi LangChain ?
Imagine que GPT-4 (ou Claude, ou Mistral) est un moteur de voiture ultra-puissant, mais nu.

Il a une intelligence énorme.
Mais il n’a pas de volant, pas de roues, pas de réservoir, et pas de phares.
LangChain, c’est le kit de construction qui permet de construire la voiture autour du moteur. C’est une bibliothèque (un ensemble d’outils) en Python et JavaScript qui aide les développeurs à connecter ces IA au monde réel.

Sans LangChain, tu dois écrire des centaines de lignes de code compliquées pour dire à l’IA « Va lire ce fichier PDF », « Souviens-toi de ce que j’ai dit avant », ou « Utilise cette calculatrice ». Avec LangChain, c’est déjà prêt à l’emploi.

![Langchain](chain.png)


## 🔩 Les 4 Piliers de LangChain
Pour bien comprendre, il faut voir LangChain comme composé de 4 grands blocs (Modules) :

1. Les Modèles () : Le CerveauModels
C’est l’endroit où tu dis quelle IA tu veux utiliser.

Exemple : « Je veux utiliser GPT-4 d’OpenAI » ou « Je veux utiliser Llama 3 de Meta ».
LangChain te donne une commande unique pour changer de cerveau sans réécrire tout ton code. C’est comme une prise universelle pour les IA.
2. Les Prompts () : Le Manuel d’InstructionsPrompts
Pour qu’une IA réponde bien, il faut lui poser la bonne question de la bonne façon.

LangChain propose des « templates » (modèles). Au lieu d’écrire , LangChain te donne des outils pour gérer ça proprement, injecter des variables et optimiser la demande.écris une blague sur {sujet}
3. Les Chaînes () : Le Moteur d’ExécutionChains
C’est le cœur du nom LangChain. C’est ce qui permet d’enchaîner les actions.

Exemple : « Prends ce texte -> Résume-le -> Traduis-le en Espagnol -> Envoie-le par email ».
Chaque étape est un maillon d’une chaîne. LangChain gère le passage des informations d’un maillon à l’autre automatiquement.
4. La Mémoire () : Le SouvenirMemory
Par défaut, les IA n’ont pas de mémoire (elles oublient ce que tu as dit juste avant).

LangChain ajoute une couche de « Mémoire ». Il peut se souvenir des 10 derniers messages, ou même résumer toute la conversation pour que l’IA garde le contexte de ce que vous êtes en train de discuter.

In [41]:
import os
# Assure-toi d'avoir installé : pip install langchain langchain-text-splitters
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    MarkdownHeaderTextSplitter,
    HTMLHeaderTextSplitter,
    PythonCodeTextSplitter
)

# ==========================================
# 🦜 COURS LANGCHAIN : LES 7 STRATÉGIES PRO
# Auteur : Crépin & Gemini
# ==========================================

def lire_fichier(chemin):
    try:
        with open(chemin, 'r', encoding='utf-8') as f: return f.read()
    except: return ""

def afficher_docs(titre, docs):
    print(f"\n🦜 MÉTHODE : {titre}")
    print(f"📊 {len(docs)} documents générés")
    for i, doc in enumerate(docs[:3]): # On affiche que les 3 premiers pour pas polluer
        preview = doc.page_content[:60].replace("\n", " ")
        print(f"   🔹 Chunk {i+1}: {preview}...")
    if len(docs) > 3: print("   ... (suite masquée)")


## 1. Taille fixe (Version Pro)
L’explication : C’est exactement la même chose que dans la partie 1 (couper tout les X caractères), mais codée par des professionnels. C’est garanti d’être plus rapide et sans bugs.


In [42]:


md_text = lire_fichier("data/histoire_ia.md")
html_text = lire_fichier("data/page_web.html")

# 1️⃣ FIXED-SIZE
# separator="" force la coupe brutale au caractère
splitter_1 = CharacterTextSplitter(separator="", chunk_size=200, chunk_overlap=0)
afficher_docs("1. FIXED-SIZE", splitter_1.create_documents([md_text]))



🦜 MÉTHODE : 1. FIXED-SIZE
📊 8 documents générés
   🔹 Chunk 1: L'Histoire et l'Évolution de l'Intelligence Artificielle  1....
   🔹 Chunk 2: and Intelligence". Il y pose la question fondamentale : "Les...
   🔹 Chunk 3: attre un champion d'échecs en quelques années. C'était l'épo...
   ... (suite masquée)



##  2. Chevauchement (Version Pro)
L’explication : Ici, on ajoute le . C’est le même principe que les photocopies avec chevauchement, mais LangChain gère les calculs pour nous.chunk_overlap=50


In [43]:

# 2️⃣ OVERLAPPING
splitter_2 = CharacterTextSplitter(separator="", chunk_size=200, chunk_overlap=50)
afficher_docs("2. OVERLAPPING", splitter_2.create_documents([md_text]))



🦜 MÉTHODE : 2. OVERLAPPING
📊 10 documents générés
   🔹 Chunk 1: L'Histoire et l'Évolution de l'Intelligence Artificielle  1....
   🔹 Chunk 2: Turing et son célèbre article "Computing Machinery and Intel...
   🔹 Chunk 3: s les années qui suivirent, l'optimisme était immense. On pe...
   ... (suite masquée)



##  3. Sémantique (Version Pro)
L’explication : On demande à LangChain : « Coupe uniquement aux points (.) ». Cela transforme notre texte en une liste de phrases. C’est très simple à demander avec un outil pro, alors que c’était compliqué à coder à la main.


In [44]:

# 3️⃣ SEMANTIC (Phrases)
# On force le split uniquement sur les points "."
splitter_3 = RecursiveCharacterTextSplitter(separators=["."], chunk_size=1000, chunk_overlap=0)
afficher_docs("3. SEMANTIC (Phrases)", splitter_3.create_documents([md_text]))



🦜 MÉTHODE : 3. SEMANTIC (Phrases)
📊 2 documents générés
   🔹 Chunk 1: L'Histoire et l'Évolution de l'Intelligence Artificielle  1....
   🔹 Chunk 2: .  2010+ : L'avènement du Deep Learning (Réseaux de neurones...



##  4. Hiérarchique / Markdown (La puissance du Pro)
L’explication : Ici, on voit la vraie puissance des outils pro. Non seulement ça coupe le texte selon les titres (, ), mais en plus, ça colle une étiquette sur chaque morceau.
Le morceau de texte dir###« Je suis le texte sur l’IA... (Étiquette : Je viens du Chapitre 2) ». C’est vital pour organiser une base de connaissances géante.


In [45]:


# 4️⃣ HIERARCHICAL (Markdown)
headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
splitter_4 = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# Note : Markdown splitter retourne des docs avec métadonnées, pas besoin de create_documents
docs_4 = splitter_4.split_text(md_text)
print(f"\n🦜 MÉTHODE : 4. HIERARCHICAL (Markdown)")
print(f"📊 {len(docs_4)} sections trouvées")
for doc in docs_4:
    print(f"   🔹 Contenu: {doc.page_content[:30]}... | Métadata: {doc.metadata}")



🦜 MÉTHODE : 4. HIERARCHICAL (Markdown)
📊 1 sections trouvées
   🔹 Contenu: L'Histoire et l'Évolution de l... | Métadata: {}



##  5. Récursif (Le Standard de l’Industrie)
L’explication : C’est l’outil le plus utilisé au monde pour le RAG (les systèmes qui répondent à tes questions). On lui dit : « Fais des morceaux de 300 caractères, essaie de garder les paragraphes ensemble, et mets 50 caractères de sécurité entre chaque ». C’est le meilleur compromis jamais inventé.


In [46]:


# 5️⃣ RECURSIVE (Le Standard)
splitter_5 = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
afficher_docs("5. RECURSIVE (Standard RAG)", splitter_5.create_documents([md_text]))





🦜 MÉTHODE : 5. RECURSIVE (Standard RAG)
📊 6 documents générés
   🔹 Chunk 1: L'Histoire et l'Évolution de l'Intelligence Artificielle  1....
   🔹 Chunk 2: Dans les années qui suivirent, l'optimisme était immense. On...
   🔹 Chunk 3: L'Hiver de l'IA  Cependant, vers les années 70, les financem...
   ... (suite masquée)



##  6. Adaptatif (Version Pro)
L’explication : C’est la même logique intelligente qu’avant (si HTML -> fais ça, si texte -> fais ça), mais on assemble des blocs


In [47]:


# 6️⃣ ADAPTIVE (Logique Python + LangChain)
def adaptive_splitter(text):
    if "<html>" in text:
        return HTMLHeaderTextSplitter(headers_to_split_on=[("h1", "Titre")]).split_text(text)
    elif len(text) > 1000:
        return RecursiveCharacterTextSplitter(chunk_size=500).create_documents([text])
    else:
        return CharacterTextSplitter(chunk_size=100).create_documents([text])

print("\n🦜 MÉTHODE : 6. ADAPTIVE")
print("   -> Test sur HTML :")
afficher_docs("Adaptive (HTML)", adaptive_splitter(html_text))
print("   -> Test sur Texte Long :")
afficher_docs("Adaptive (MD)", adaptive_splitter(md_text))



🦜 MÉTHODE : 6. ADAPTIVE
   -> Test sur HTML :

🦜 MÉTHODE : Adaptive (HTML)
📊 2 documents générés
   🔹 Chunk 1: L'Intelligence Artificielle Web...
   🔹 Chunk 2: L'IA change le web. C'est une révolution structurelle.   Les...
   -> Test sur Texte Long :

🦜 MÉTHODE : Adaptive (MD)
📊 5 documents générés
   🔹 Chunk 1: L'Histoire et l'Évolution de l'Intelligence Artificielle  1....
   🔹 Chunk 2: Dans les années qui suivirent, l'optimisme était immense. On...
   🔹 Chunk 3: 2. Le Réveil : Machine Learning et Big Data (1990-2010)  Ave...
   ... (suite masquée)



##  7. HTML structurel (Version Pro)
L’explication : Au lieu d’utiliser des expressions régulières (un peu risquées et complexes pour couper du HTML), on utilise . C’est un outil spécialisé qui comprend parfaitement le langage des sites web et sépare le titre H1 du titre H2 proprement, sans se mélanger les pinceaux.HTMLHeaderTextSplitter

In [48]:

# 7️⃣ STRUCTURAL (HTML)
# HTMLHeaderTextSplitter s'attend à des balises de type h1..h6 (ex: 'h1', 'h2').
# 'div' n'est pas un en-tête et provoquait l'erreur int('iv').
headers_html = [("h1", "Titre Principal"), ("h2", "Sous-titre"), ("h3", "Section")]
splitter_7 = HTMLHeaderTextSplitter(headers_to_split_on=headers_html)
docs_7 = splitter_7.split_text(html_text)
print(f"\n🦜 MÉTHODE : 7. STRUCTURAL (HTML)")
for doc in docs_7:
    print(f"   🔹 {doc.page_content[:40]}... | {doc.metadata}")


🦜 MÉTHODE : 7. STRUCTURAL (HTML)
   🔹 L'Intelligence Artificielle Web... | {'Titre Principal': "L'Intelligence Artificielle Web"}
   🔹 L'IA change le web. C'est une révolution... | {'Titre Principal': "L'Intelligence Artificielle Web"}
   🔹 Les technologies... | {'Titre Principal': "L'Intelligence Artificielle Web", 'Sous-titre': 'Les technologies'}
   🔹 Le RAG permet d'interroger ces pages HTM... | {'Titre Principal': "L'Intelligence Artificielle Web", 'Sous-titre': 'Les technologies'}
